## --- Day 4: Ceres Search ---
"Looks like the Chief's not here. Next!" One of The Historians pulls out a device and pushes the only button on it. After a brief flash, you recognize the interior of the Ceres monitoring station!

As the search for the Chief continues, a small Elf who lives on the station tugs on your shirt; she'd like to know if you could help her with her word search (your puzzle input). She only has to find one word: XMAS.

This word search allows words to be horizontal, vertical, diagonal, written backwards, or even overlapping other words. It's a little unusual, though, as you don't merely need to find one instance of XMAS - you need to find all of them. Here are a few ways XMAS might appear, where irrelevant characters have been replaced with .:

```
..X...
.SAMX.
.A..A.
XMAS.S
.X....
```
The actual word search will be full of letters instead. For example:
```
MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX
```
In this word search, XMAS occurs a total of 18 times; here's the same word search again, but where letters not involved in any XMAS have been replaced with .:
```
....XXMAS.
.SAMXMS...
...S..A...
..A.A.MS.X
XMASAMX.MM
X.....XA.A
S.S.S.S.SS
.A.A.A.A.A
..M.M.M.MM
.X.X.XMASX
```
Take a look at the little Elf's word search. How many times does XMAS appear?

In [2]:
# dont know if thats easy with regex.
# thinking 
# 1 produce all possible strings
 #   hori
 #   vert
 #   diag
 #   rev_diag
 #   overlap
# 2 run re.findall and count

with open("i4.txt") as f:
    lines = [line.strip() for line in f.readlines()]

size = len(lines)
size

140

In [3]:
def create_diag(lines):
    diag=[]
    for i in range(len(lines)): # 0-139
        diag.append("".join(lines[i-j][size-j-1] for j in range(i+1)))
    return diag
        
        
create_diag(lines[:10])

['S',
 'SS',
 'SAM',
 'MMXS',
 'XXMMA',
 'MASXAM',
 'XMMXMSX',
 'MSSXSAXX',
 'XXAXAAXMM',
 'SAMASMSXAM']

its actually enough to create this part of the diags, cause all remaining diag parts can be produced by using a rotated version of the lines.
A rotation in turn can be thought of as a flip and transposition, so the 4 horizontal matrices are actually also the rotated versions. 

actually not quite, flipping the lines does not make it a 180° rotation, for that, the lines also need to be reversed.

maybe just do the rotations 

In [4]:
def rotate(lines):
    size=len(lines)
    rotated=[[] for i in range(size)]
    for i in range(size):
        for j in range(len(lines)):
            rotated[j].insert(0,lines[i][j])
    return ["".join(rotated[i]) for i in range(size)]


In [ ]:

all_strings = []
for i in range(4):
    all_strings.append(lines)
    diags = create_diag(lines)
    if i < 2:
        # omit the longest diag for the first 2 rotations to not include it twice
        all_strings.append(diags[:-1]) 
        all_strings.append([diag[::-1] for diag in diags[:-1]]) # reverse
    else:
        all_strings.append(diags) 
        all_strings.append([diag[::-1] for diag in diags])
    lines = rotate(lines)
    


In [6]:
import re
count = 0
for matrix in all_strings:
    for line in matrix:
        #print(line)
        count += len(re.findall("XMAS", line))
        
count

2569

## --- Part Two ---
The Elf looks quizzically at you. Did you misunderstand the assignment?

Looking for the instructions, you flip over the word search to find that this isn't actually an XMAS puzzle; it's an X-MAS puzzle in which you're supposed to find two MAS in the shape of an X. One way to achieve that is like this:
```
M.S
.A.
M.S
```	
Irrelevant characters have again been replaced with . in the above diagram. Within the X, each MAS can be written forwards or backwards.

Here's the same example from before, but this time all of the X-MASes have been kept instead:
```	
.M.S......
..A..MSMS.
.M.S.MAA..
..A.ASMSM.
.M.S.M....
..........
S.S.S.S.S.
.A.A.A.A..
M.M.M.M.M.
..........
```	
In this example, an X-MAS appears 9 times.

Flip the word search from the instructions back over to the word search side and try again. How many times does an X-MAS appear?

FAAAAACK, so now it turs to: 
- for all As
- for half of all 8 orientations (0, 1), (1,1)
- check for a cross, checking with the other half of missing orientations 

In [28]:
test = [line[:10] for line in lines[:10]]
test

['XXASMSMSXM',
 'ASMMAAAMAM',
 'XSAMSMMSAM',
 'AMAMXAASMM',
 'XSAMMMMXAX',
 'MSASXMXSSM',
 'ASXMASAAAX',
 'XMXSAXMXMA',
 'MSMMASMAMX',
 'AAAAMXXAMX']

In [73]:
with open("i4.txt") as f:
    lines = [line.strip() for line in f.readlines()]

size = len(lines)
size

140

In [81]:

directions = [ (1, 1), (1, 0), (1, -1),(0, -1), (-1, -1),  (-1, 0), (-1, 1), (0, 1)]
def get_mas(spot, matrix, direction):
    x,y = spot
    dx, dy = direction
    size = len(matrix)
    mas = []
    for i in [-1,0,1]:
        mas.append(matrix[x+i*dx][y+i*dy])
    return "".join(mas)

for direc in directions:
    print(get_mas((1,1), test, direc))



XSA
XSS
ASX
MSA
ASX
SSX
XSA
ASM


In [ ]:
def has_x(spot, matrix):
    # spot is a tuple (i,j)
    x,y = spot
    if matrix[x][y] != "A": return False
    for i, direction in enumerate(directions):
        mas = get_mas(spot, matrix, direction)
        if mas != "MAS": continue
        if i%2 == 1: continue # no straight (wich would be plus, only x-s)
        if get_mas(spot, matrix, directions[(i+2)%8]) == "MAS": return True
        if get_mas(spot, matrix, directions[(i-2)%8]) == "MAS": return True
    return False
        
i, j = 1,4
print(lines[i-1][j-1:j+2])
print(lines[i][j-1:j+2])
print(lines[i+1][j-1:j+2])
has_x((i,j), test)

SMS
MAA
MSM


True

In [65]:
size

140

In [92]:
tcounter = 0
for i in range(1,size-1):
    for j in range(1,size-1):
        spot = (i, j)
        if has_x(spot, lines):
            # print(spot)
            # print(lines[i-1][j-1:j+2])
            # print(lines[i][j-1:j+2])
            # print(lines[i+1][j-1:j+2])
            
            tcounter += 1
        
tcounter

1998